# Tutorial for OTP Features in MLCBase

[![PyPI](https://img.shields.io/pypi/v/mlcbase)](https://pypi.org/project/mlcbase/) &nbsp;
[![license](https://img.shields.io/github/license/wmchen/mlcbase.svg)](https://www.apache.org/licenses/LICENSE-2.0)

Author: [Weiming Chen](https://weimingchen.net) and [Yuanshuang Sun](https://www.mulingcloud.com/author/yuanshuang-sun/)

## Introduction

We support two methods for OTP: Time-based One-Time Password (TOTP) and HMAC-based One-Time Password (HOTP).

In [1]:
import sys
sys.path.append("../src")
from datetime import datetime, timedelta
from mlcbase import Logger, generate_otp_secret, generate_otp_code, verify_otp_code

logger = Logger()
logger.init_logger()


👋 Welcome to use MuLingCloud. We aim to let everything easier.

📍 mlcbase (1.2.0.dev.202405) imported



## 1. Generate OTP secret

You can generate an OTP secret by calling `generate_otp_secret()`.

#### Arguments

| args | type | remark |
| :--- | :--- | :----- |
| `account_name` | str | The name of the account associated with the key |
| `method` | str | The OTP method. Options including "TOTP" and "HOTP". Defaults to "TOTP" |
| `issuer` | str | The name of the key's issuing organization. Defaults to "MuLingCloud" |
| `algorithm` | str | The hashing algorithm used to generate the OTP code. Options including "SHA1", "SHA256", and "SHA512". Defaults to "SHA1" |
| `period` | int | The length of time in seconds used to generate a counter for the TOTP code calculation. Only be used in TOTP method. Defaults to 30 |
| `initial_count` | int | The starting HMAC counter value. Only be used in HOTP method. Defaults to 0 |
| `digits` | int | The number of digits in the generated OTP code, which should be 6 or 8. Defaults to 6 |
| `return_secret_key` | bool | Whether to return the value of secret key. Defaults to True |
| `return_qr_code` | bool | Whether to return the QR code in base64 format. Defaults to True |
| `qr_save_path` | Optional[PathLikeType] | The path to save the QR code. Defaults to None |
| `qr_config` | dict | The configuration of the QR code |
| `logger` | Optional[Logger] | Defaults to None |

The default `qr_config` is as follows. The details of QRCode arguments plese refer to https://github.com/lincolnloop/python-qrcode.

If `has_logo` is True but `logo` is None, it will attach the logo of ours automatically.

```python
qr_config = ConfigDict(
    version=1,                         # QRCode arguments
    error_correction=ERROR_CORRECT_M,  # QRCode arguments
    box_size=10,                       # QRCode arguments
    border=4,                          # QRCode arguments
    fit=True,                          # QRCode arguments
    back_color="white",                # The background color of the QRCode
    fill_color="black",                # The foreground color of the QRCode
    has_logo=True,                     # whether to attach a logo in the center of QRCode
    logo=None,                         # The local path or the url of the logo
    factor=4                           # The size divisor of the logo
)
```

#### Return

It returns a ConfigDict containing the meta data, secret key (optional), and QR code (optional)

### 1.1 TOTP method

The example as follows use the default setting that generates an TOTP secret which can be parsed by Google Authenticator.

You can use your Google Authenticator to scan the [QRCode](./examples/totp_qr_code.png) and try.

In [2]:
totp_secret = generate_otp_secret(account_name="your account name", qr_save_path="./examples/totp_qr_code.png", logger=logger)
totp_secret

{'metadata': {'method': 'TOTP',
  'issuer': 'MuLingCloud',
  'account_name': 'your account name',
  'algorithm': 'SHA1',
  'period': 30,
  'digits': 6},
 'url': 'otpauth://totp/MuLingCloud:your%20account%20name?secret=BMQX532SW2KAGJ66QKGYSH5BOFSUBYHO&issuer=MuLingCloud',
 'secret': 'BMQX532SW2KAGJ66QKGYSH5BOFSUBYHO',
 'qr_code': b'iVBORw0KGgoAAAANSUhEUgAAAeoAAAHqCAYAAADLbQ06AAAV3ElEQVR4nO3dO5YcR3YG4GwdrGK2MMboAZ/dhszxJFP0KHIB9NHwZwECx9N44Hg0aaDhY4ZDg1vgNkoGWVCjWI/Iqqi8f0V+3zl9gAN0Z0Q+/46syJt3m81mMwEAkf6pugMAwGGCGgCCCWoACCaoASCYoAaAYIIaAIIJagAIJqgBIJigBoBgghoAgglqAAgmqAEgmKAGgGCCGgCCCWoACCaoASCYoAaAYIIaAIIJagAIJqgBIJigBoBgghoAgglqAAgmqAEgmKAGgGCCGgCCCWoACCaoASCYoAaAYIIaAIIJagAIJqgBIJigBoBgghoAgglqAAgmqAEgmKAGgGCCGgCCCWoACPaiugOn3N3dVXch0mazafq+3tuvtd1W6ft3lO3cynpcpvfxsrbzrUrVcdrKiBoAgglqAAgmqAEgmKAGgGCCGgCCCWoACCaoASCYoAaAYIIaAILFVyZrlV5ZptUolYNGqQzVu4JU73Z7S6+E1bt/VeubXsHM9TSLETUABBPUABBMUANAMEENAMEENQAEE9QAEExQA0AwQQ0AwQQ1AAQbpjJZq1EqPlW1m155qaqSU9V69FZVSSy9Ulyr9OO0t7VdT6sYUQNAMEENAME

### 1.2 HOTP method

The usage of HOTP method is similar to that of TOTP.

The example as follows generates an HOTP secret with `initial_count=113`.

In [3]:
hotp_secret = generate_otp_secret(account_name="your account name", method="HOTP", initial_count=113, logger=logger)
hotp_secret

{'metadata': {'method': 'HOTP',
  'issuer': 'MuLingCloud',
  'account_name': 'your account name',
  'algorithm': 'SHA1',
  'initial_count': 113,
  'digits': 6},
 'url': 'otpauth://hotp/MuLingCloud:your%20account%20name?secret=X7YHAB3HCFE6PD3CDYAABXH6MHZRJJOE&issuer=MuLingCloud&counter=113',
 'secret': 'X7YHAB3HCFE6PD3CDYAABXH6MHZRJJOE',
 'qr_code': b'iVBORw0KGgoAAAANSUhEUgAAAhIAAAISCAYAAACZPSa/AAAYvUlEQVR4nO3dPXYtV5kG4FIvj4IpOOgL7QFIAaEzcNZkxgzA+b3OGUC7nTWZIXNIIJEbEIGnwDTUgZE5V/ccnTpv7VP7q13Ps5YXBqSqXf+vvqr66ubp6elpAgAI/EfvAQAA2yVIAAAxQQIAiAkSAEBMkAAAYoIEABATJACAmCABAMQECQAgJkgAADFBAgCICRIAQEyQAABiggQAEBMkAICYIAEAxAQJACAmSAAAMUECAIgJEgBATJAAAGKCBAAQEyQAgJggAQDEBAkAICZIAAAxQQIAiAkSAEBMkAAAYoIEABATJACAmCABAMQECQAgJkgAADFBAgCICRIAQEyQAABiggQAEBMkAIDYR70HcM7NzU3vIZT09PQ06+dar7/q821t7nL0Wi9ztV5/rddLr/lWn151oxwf1VXfX1QkAICYIAEAxAQJACAmSAAAMUECAIgJEgBATJAAAGKCBAAQEyQAgFj5zpZzVe/8NVf1zofVO1ZW74w3yvJWX47q0+s131E6dDrf16IiAQDEBAkAICZIAAAxQQIAiAkSAEBMkAAAYoIEABATJACAmCABAMSG6Ww51yidAPc239ad9np17ms

## 2. Generate OTP code

You can generate an OTP code by calling `generate_otp_code()`.

#### Arguments

| args | type | remark |
| :--- | :--- | :----- |
| `secret_key` | str | The secret key of OTP |
| `count` | Optional[int] | The current count number. Only be used in HOTP method. Defaults to None |
| `method` | str | The OTP method. Options including "TOTP" and "HOTP". Defaults to "TOTP" |
| `algorithm` | str | The hashing algorithm used to generate the OTP code. Options including "SHA1", "SHA256", and "SHA512". Defaults to "SHA1" |
| `period` | int | The length of time in seconds used to generate a counter for the TOTP code calculation. Only be used in TOTP method. Defaults to 30 |
| `initial_count` | int | The starting HMAC counter value. Only be used in HOTP method. Defaults to 0 |
| `digits` | int | The number of digits in the generated OTP code, which should be 6 or 8. Defaults to 6 |
| `logger` | Optional[Logger] | Defaults to None |

#### Return

It returns the current OTP code in `str`.

### 2.1 TOTP method

In [4]:
totp_code = generate_otp_code(secret_key=totp_secret.secret, 
                              method="TOTP",
                              algorithm=totp_secret.metadata.algorithm,
                              period=totp_secret.metadata.period,
                              digits=totp_secret.metadata.digits,
                              logger=logger)
totp_code

'212820'

### 2.2 HOTP method

In [5]:
hotp_code = generate_otp_code(secret_key=hotp_secret.secret, 
                              count=150,
                              method="HOTP",
                              algorithm=hotp_secret.metadata.algorithm,
                              initial_count=hotp_secret.metadata.initial_count,
                              digits=hotp_secret.metadata.digits,
                              logger=logger)
hotp_code

'991115'

## 3. Verify OTP code

You can verify a code by calling `verify_otp_code()`.

#### Arguments

| args | type | remark |
| :--- | :--- | :----- |
| `code` | str | The code waiting for verification |
| `secret_key` | str | The secret key of OTP |
| `count` | Optional[int] | The current count number. Only be used in HOTP method. Defaults to None |
| `method` | str | The OTP method. Options including "TOTP" and "HOTP". Defaults to "TOTP" |
| `algorithm` | str | The hashing algorithm used to generate the OTP code. Options including "SHA1", "SHA256", and "SHA512". Defaults to "SHA1" |
| `period` | int | The length of time in seconds used to generate a counter for the TOTP code calculation. Only be used in TOTP method. Defaults to 30 |
| `initial_count` | int | The starting HMAC counter value. Only be used in HOTP method. Defaults to 0 |
| `digits` | int | The number of digits in the generated OTP code, which should be 6 or 8. Defaults to 6 |
| `for_time` | Optional[datetime] | Time to check OTP at (defaults to now). Only used for TOTP method. Defaults to None |
| `valid_window` | int | Extends the validity to this many counter ticks before and after the current one. Only used for TOTP method. Defaults to 0 |
| `logger` | Optional[Logger] | Defaults to None |

#### Return

It returns True if verification succeeded, otherwise returns False

### 3.1 TOTP method

In [6]:
verify_otp_code(totp_code,
                secret_key=totp_secret.secret,
                method="TOTP",
                algorithm=totp_secret.metadata.algorithm,
                period=totp_secret.metadata.period,
                digits=totp_secret.metadata.digits,
                logger=logger)

True

You can set `for_time` to generate the code based on the specific time. If `for_time` is None, it generates the code based on now time.

You can set `valid_window` to extend the validation window of TOTP method.

In [7]:
last_time = datetime.now() - timedelta(seconds=30)

In [8]:
verify_otp_code(totp_code,
                secret_key=totp_secret.secret,
                method="TOTP",
                algorithm=totp_secret.metadata.algorithm,
                period=totp_secret.metadata.period,
                digits=totp_secret.metadata.digits,
                for_time=last_time,
                logger=logger)

False

In [9]:
verify_otp_code(totp_code,
                secret_key=totp_secret.secret,
                method="TOTP",
                algorithm=totp_secret.metadata.algorithm,
                period=totp_secret.metadata.period,
                digits=totp_secret.metadata.digits,
                for_time=last_time,
                valid_window=1,
                logger=logger)

True

### 3.2 HOTP method

In [10]:
verify_otp_code(hotp_code,
                secret_key=hotp_secret.secret,
                count=150,
                method="HOTP",
                algorithm=hotp_secret.metadata.algorithm,
                initial_count=hotp_secret.metadata.initial_count,
                digits=hotp_secret.metadata.digits,
                logger=logger)

True